In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
"""
!pip install comet-ml
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="5BHASx2Q9HBWhSzOPryQC92zu",
    project_name="fake-news-detection",
    workspace="nikesh")
"""

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report

#!pip install transformers &> /dev/null
!pip install sentence-transformers &> /dev/null
#from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

!pip install pytorch-lightning &> /dev/null
import pytorch_lightning as plt

Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Dataset Loading

In [3]:
HEADLINE_COLUMN = "Headline"
BODY_COLUMN = "articleBody"
STANCE_COLUMN = "Stance"

categories = {"agree":0,"disagree":1,"discuss":2,"unrelated":3}

train_bodies_csv = pd.read_csv("/content/drive/My Drive/Uni Work/train_bodies.csv")
train_stances_csv = pd.read_csv("/content/drive/My Drive/Uni Work/train_stances.csv")
train_dataset = train_stances_csv.merge(train_bodies_csv, on='Body ID', how="inner")

train_headlines = train_dataset[HEADLINE_COLUMN].values
train_bodies = train_dataset[BODY_COLUMN].values
train_truth = np.array([categories[stance] for stance in train_dataset[STANCE_COLUMN].values])
train_related_truth = np.array([int(stance<3) for stance in train_truth]) # unrelated = 0, related = 1

test_bodies_csv = pd.read_csv("/content/drive/My Drive/Uni Work/competition_test_bodies.csv")
test_stances_csv = pd.read_csv("/content/drive/My Drive/Uni Work/competition_test_stances.csv")
test_dataset = test_stances_csv.merge(test_bodies_csv, on='Body ID', how="inner")

test_headlines = test_dataset[HEADLINE_COLUMN].values
test_bodies = test_dataset[BODY_COLUMN].values
test_truth = np.array([categories[stance] for stance in test_dataset[STANCE_COLUMN].values])
test_related_truth = np.array([int(stance<3) for stance in test_truth]) # unrelated = 0, related = 1

train_bodies_csv = train_stances_csv = test_bodies_csv = test_stances_csv = None # free memory

# Data Cleaning

In [4]:
stop = set(stopwords.words('english'))

# from practicals
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    
    return url.sub('', text)

# from practicals
def remove_html(text):
    html = re.compile(r'<.*?>')
    
    return html.sub('', text)

# from practicals
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)

def data_cleanup(data):
    x = list(map(remove_URL, data))
    x = list(map(remove_html, data))
    x = list(map(remove_emoji, data))
    x = np.array(x)
    return x

#train_headlines = data_cleanup(train_headlines)
#train_bodies = data_cleanup(train_bodies)
#test_headlines = data_cleanup(test_headlines)
#test_bodies = data_cleanup(test_bodies)

# Feature Extraction

## TF-IDF

In [ ]:
# tf-idf feature extraction 
train_tfidf_data = np.concatenate((np.unique(train_headlines), np.unique(train_bodies)))
tfidf = TfidfVectorizer(decode_error='ignore', lowercase=True, min_df=2, stop_words=stop, max_features=10000) 
train_tfidf = tfidf.fit(train_tfidf_data.astype('U')) 
tfidf_bodies = tfidf.transform(train_bodies.astype('U'))
tfidf_headlines = tfidf.transform(train_headlines.astype('U'))

tfidf_test_bodies = tfidf.transform(test_bodies.astype('U'))
tfidf_test_headlines = tfidf.transform(test_headlines.astype('U'))

In [ ]:
np.shape(tfidf_bodies)

(49972, 3299)

## S-BERT

In [5]:
def remove_stop_words(string):
    split_words = string.split()
    return " ".join([word for word in split_words if word.lower() not in stop])

for index, (headline, body) in enumerate(zip(train_headlines, train_bodies)):
    train_headlines[index] = remove_stop_words(headline)
    train_bodies[index] = remove_stop_words(body)

for index, (test_headline, test_body) in enumerate(zip(test_headlines, test_bodies)):
    test_headlines[index] = remove_stop_words(test_headline)
    test_bodies[index] = remove_stop_words(test_body)

sbert_encoder = SentenceTransformer("paraphrase-MiniLM-L6-v2")

sbert_bodies = sbert_encoder.encode(train_bodies, show_progress_bar=True)
sbert_headlines = sbert_encoder.encode(train_headlines, show_progress_bar=True)

sbert_test_bodies = sbert_encoder.encode(test_bodies, show_progress_bar=True)
sbert_test_headlines = sbert_encoder.encode(test_headlines, show_progress_bar=True)

You try to use a model that was created with version 1.2.0, however, your version is 1.1.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [ ]:
print(np.shape(sbert_bodies))
print(np.shape(sbert_headlines))

#2ai) Machine Learning Method: Logistic Regression

##TF-IDF Embedding

In [ ]:
tfidf_similarity_vectors = []
for headline, body in zip(tfidf_headlines, tfidf_bodies):
    tfidf_similarity_vectors.append(cosine_similarity(headline, body)[0])
tfidf_similarity_vectors = np.array(tfidf_similarity_vectors)

model = LogisticRegression().fit(tfidf_similarity_vectors, train_related_truth)
train_predictions = model.predict(tfidf_similarity_vectors)
print("*** Train Metrics ***")
print(classification_report(train_related_truth, train_predictions))
print("*** ************* ***")
# test set
tfidf_similarity_vectors = []
for headline, body in zip(tfidf_test_headlines, tfidf_test_bodies):
    tfidf_similarity_vectors.append(cosine_similarity(headline, body)[0])
tfidf_similarity_vectors = np.array(tfidf_similarity_vectors)

test_predictions = model.predict(tfidf_similarity_vectors)
print("*** Test Metrics ***")
print(classification_report(test_related_truth, test_predictions))
print("*** ************* ***")

KeyboardInterrupt: ignored

##SBERT Embedding

In [8]:
sbert_similarity_vectors = []
for headline, body in zip(sbert_headlines, sbert_bodies):
    sbert_similarity_vectors.append(util.pytorch_cos_sim(headline, body))
sbert_similarity_vectors = np.array(sbert_similarity_vectors)

model = LogisticRegression().fit(sbert_similarity_vectors.reshape(-1,1), train_related_truth)
train_predictions = model.predict(sbert_similarity_vectors.reshape(-1,1))
print("********************** Train Metrics **********************")
print(classification_report(train_related_truth, train_predictions))
print("***********************************************************")
# test set
sbert_similarity_vectors = []
for headline, body in zip(sbert_test_headlines, sbert_test_bodies):
    sbert_similarity_vectors.append(util.pytorch_cos_sim(headline, body))
sbert_similarity_vectors = np.array(sbert_similarity_vectors)

test_predictions = model.predict(sbert_similarity_vectors.reshape(-1,1))
print("********************** Test  Metrics **********************")
print(classification_report(test_related_truth, test_predictions))
print("***********************************************************")

********************** Train Metrics **********************
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     36545
           1       0.95      0.93      0.94     13427

    accuracy                           0.97     49972
   macro avg       0.96      0.96      0.96     49972
weighted avg       0.97      0.97      0.97     49972

***********************************************************
********************** Test  Metrics **********************
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     18349
           1       0.96      0.95      0.96      7064

    accuracy                           0.98     25413
   macro avg       0.97      0.97      0.97     25413
weighted avg       0.98      0.98      0.98     25413

***********************************************************


In [ ]:
len(sbert_similarity_vectors)

49972

#2aii) Deep Learning Method

#2b) Deep Learning for Agree/Disagree/Discuss